In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def phases(iq: np.ndarray,start:int):
    list = []
    for i in range(start,len(iq)-2,6):
        list.append(/
            np.angle(complex(iq[i],iq[i+1]))
        )
    return np.array(list)



antenna_conf= "2-2-0-5-6"

data = pd.read_json("../folder/data.json")
data = data[data["Switch Order"] == antenna_conf]
#data = data[data["TAG"] != "B"]
#data = data[data["rssi"] >-70]
del data["level_0"]
del data["index"]
data=data.reset_index()


In [8]:
tdata = []
tlabels = []

for _, row in data.iterrows():
    for i in range(16,len(row["iq"])-6,3*2):
        tdata.append(row["iq"][i:i+6])
        tlabels.append(row["Angle"])


# On IQ samples

In [38]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, mean_squared_error

#Loading the Iris dataset

X, y = np.array(tdata), [int(x) for x in tlabels]

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBRegressor(eval_metric=mean_absolute_error)

#Training the model on the training data
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_absolute_error at 0x0000021BB4EF5A80>,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [41]:
#Making predictions on the test set
predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
print('Erro Médio Absoluto:', mae)

# Calcular MSE
mse = mean_squared_error(y_test, predictions)
print('Erro Quadrático Médio:', mse)

# Calcular RMSE
rmse = mean_squared_error(y_test, predictions, squared=False)
print('Raiz do Erro Quadrático Médio:', rmse)

Erro Médio Absoluto: 10.075640802152266
Erro Quadrático Médio: 244.23272496636068
Raiz do Erro Quadrático Médio: 15.627946921024549


In [54]:
i=np.random.randint(len(X_test))
predictions = model.predict([X_test[i]])
print(f"{predictions=} \n answer={y_test[i]}")


predictions=array([145.4424], dtype=float32) 
 answer=150


# Real Time

In [56]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import datetime


# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
def on_message(client, userdata, msg):
    data = json.loads(msg.payload)

    predictions =[]
    for i in range(16,len(data[1]["aoa"]["iq"])-6,3*2):
        
        
        predictions.append(model.predict([data[1]["aoa"]["iq"][i:i+6]]))

    print(np.mean(predictions))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)

now = datetime.datetime.now()
client.loop_forever()


Conectado com o código de resultado 0
125.93894
130.76112
127.792534
127.19378
119.932
129.11444
132.07759
121.876335
116.254295
131.12782
118.153366
117.25916
121.68928
125.639206
122.058235
122.96509
128.57399
119.090454
118.71866
146.36456
133.61725
129.00049
117.42514
143.80345
123.18994
67.55407
136.29694
117.280876
121.603226
126.734795
124.69954
127.74372
138.6247
115.8989
114.1752
118.71695
126.73897
127.11701
121.74344
126.43936
122.816986
104.12124
127.7519
139.38379
129.73293
121.96003
117.423164
133.55394
129.07379
116.893776
130.83746
130.01512
129.42128
124.7668
131.2394
124.00881
122.86719
138.62413
119.253914
129.68948
124.29162
130.92993
122.99751
126.59243
138.15448
79.41208
126.33281
106.090866
124.24592
128.44452
140.18324
125.0492
103.9159
128.94688
112.01738
128.54987
137.81549
123.37927
120.15565
111.501434
130.52264
125.85721


KeyboardInterrupt: 

16
22
28
34
40



# vector \alpha


